А теперь серьезно...

### Формат файлов

Вам выдается 4 файла:

* `train.txt` --- обучающая выборка пар запрос-документ и асессорские метки релевантности
* `test.txt` --- тестовая выборка пар запрос-документ
* `queries_test.txt` --- запросы из `train.txt`
* `queries_train.txt` --- запросы из `test.txt`

Колонки в файлах могут быть следующего типа:

* `QueryId` --- уникальный номер запроса
* `DocumentId` --- номер документа, не повторяется для одного запроса
* `DocumentLink` --- url документа
* `Relevance` --- асессорская метка релевантности

Формат файла ответов приведен ниже. Пары запрос-документ должны соответсвовать файлу `test.txt` и должны быть упорядочены по убыванию построенной функции релевантности. То есть так, как в поисковой выдаче.

Качество оценивается с помощью метрики ${NDCG}_{10}$.

In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import pandas as pd
import re

In [3]:
with open('queries_train.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

In [33]:
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import make_scorer

def dcg_score(y_true, k=10):
#     order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(y_true, y_pred, k=10):
#     lb = LabelBinarizer()
#     T = lb.fit_transform(y_true)
    actual = dcg_score(y_pred, k)
    best = dcg_score(y_true, k)
    return actual * 1. / best

In [21]:
def word_neighbours(text, n=5):
    words = []
    text = [re.sub('[,.?!"":;+={}()&*^%$#@/]', '', t).lower() for t in text.split()]
    for word in text:
        for i in range(n, len(word)):
            words.append(word[:i])
    return words

In [6]:
with open('queries_test.txt') as f:
    queries = f.readlines()
queries = list(map(lambda x: x[:-1].split('\t')[1], queries))

In [38]:
score = []
with open('train.txt') as ft:
    with open('submission.txt', 'w') as fb:
        ft.readline()
        fb.write('QueryId,DocumentId\n')

        qids, docids, relevs, true_relevs = [], [], [], []
        for line in ft:
            split = line.split(',')
            rel, qid, docid = int(split[0]), int(split[1]), int(split[2])
            url = ''.join(split[3:])
            qid, docid = int(qid), int(docid)
            true_relevs.append(rel)
            
            if len(qids) > 0 and qid != qids[-1]:
                
                order = np.argsort(relevs)[::-1]
                docids = np.array(docids)[order]
#                 print(ndcg_score(true_relevs, np.array(true_relevs)[order]))
                score.append(ndcg_score(true_relevs, np.array(true_relevs)[order]))

                for i in range(len(qids)):
                    fb.write('{},{}\n'.format(qids[i], docids[i]))
                    
                qids, docids, relevs, true_relevs = [], [], [], []
            try:
                page = urlopen(url, timeout=10)
                soup = BeautifulSoup(page, "html.parser")
                for elem in soup.find_all(['style', 'script', '[document]', 'head', 'title']):
                    elem.extract()
#                 text = word_neighbours(' '.join(BeautifulSoup(urlopen(url, timeout=10), 'lxml').stripped_strings))
                text = word_neighbours(soup.getText())
            except OSError:
                continue
            frec = 0
            for word in word_neighbours(queries[qid - 1]):
                frec += np.mean([t == word for t in text])
                
            break
            qids.append(qid)
            docids.append(docid)
            relevs.append(frec)
            
print(np.mean(score))

0.37013591549


In [40]:
score = []
with open('test.txt') as ft:
    with open('submission.txt', 'w') as fb:
        ft.readline()
        fb.write('QueryId,DocumentId\n')

        qids, docids, relevs = [], [], []
        counter = 0
        for line in ft:
            split = line.split(',')
            qid, docid = int(split[0]), int(split[1])
            url = ''.join(split[2:])
            qid, docid = int(qid), int(docid)
            
            if len(qids) > 0 and qid != qids[-1]:
                order = np.argsort(relevs)[::-1]
                docids = np.array(docids)[order]
                print(counter)
                counter += 1
                
                for i in range(len(qids)):
                    fb.write('{},{}\n'.format(qids[i], docids[i]))
                    
                qids, docids, relevs = [], [], []
            try:
                page = urlopen(url, timeout=10)
                soup = BeautifulSoup(page, "html.parser")
                for elem in soup.find_all(['style', 'script', '[document]', 'head', 'title']):
                    elem.extract()
#                 text = word_neighbours(' '.join(BeautifulSoup(urlopen(url, timeout=10), 'lxml').stripped_strings))
                text = word_neighbours(soup.getText())
            except OSError:
                continue
            frec = 0
            for word in word_neighbours(queries[qid - 101]):
                frec += np.mean([t == word for t in text])
                
            qids.append(qid)
            docids.append(docid)
            relevs.append(frec)